# **Data Readiness For AI Checklist - Part 5**

 * Creator(s) John Pill
 * Affiliation: UK Met Office
 * History: 1.0
 * Last update: 27 August 2024.


---

## **Tutorial Material**

* **Run this Jupyter notebook locally using Jupyter Lab**
* **Select 'Run All Cells' from the 'Run' menu to generate the checklist**.
* **Remember to save your notebook regularly as you work through it.**


## **Data section, optional**
Scripts for pulling the data into the notebook assuming

---

## **Setup Notebook**

In [14]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import sys
import os
import numpy as np
from aidatareadiness import utils
from aidatareadiness.utils import WIDGET_WIDTH, DESCRIPTION_STYLE, PLACEHOLDER  
from aidatareadiness.checklist_auto import gridded 

## **Load Data**

In [15]:
# Use the following function to laod your dataset and check its file format is compatible. 

gridded_file_path = "HadCRUT.5.0.2.0.analysis.anomalies.1.nc"
dataset = gridded.detect_gridded_format_and_open(gridded_file_path)
dataset

<xarray.Dataset> Size: 44MB
Dimensions:         (time: 2098, latitude: 36, longitude: 72, bnds: 2)
Coordinates:
  * time            (time) datetime64[ns] 17kB 1850-01-16T12:00:00 ... 2024-1...
  * latitude        (latitude) float64 288B -87.5 -82.5 -77.5 ... 77.5 82.5 87.5
  * longitude       (longitude) float64 576B -177.5 -172.5 ... 172.5 177.5
    realization     int64 8B ...
Dimensions without coordinates: bnds
Data variables:
    tas             (time, latitude, longitude) float64 44MB ...
    time_bnds       (time, bnds) datetime64[ns] 34kB ...
    latitude_bnds   (latitude, bnds) float64 576B ...
    longitude_bnds  (longitude, bnds) float64 1kB ...
Attributes:
    comment:      2m air temperature over land blended with sea water tempera...
    history:      Data set built at: 2024-12-06T14:56:47+00:00
    institution:  Met Office Hadley Centre / Climatic Research Unit, Universi...
    licence:      HadCRUT5 is licensed under the Open Government Licence v3.0...
    reference:    C. P. Morice, J. J. Kennedy, N. A. Rayner, J. P. Winn, E. H...
    source:       CRUTEM.5.0.2.0 HadSST.4.0.1.0
    title:        HadCRUT.5.0.2.0 blended land air temperature and sea-surfac...
    version:      HadCRUT.5.0.2.0
    Conventions:  CF-1.7

In [16]:
# Load checklist from JSON file:
checklist = utils.load_checklist()

#### Reset stored answers to start again:

In [17]:
# Reset all checklist answers back to original blank answers for all sections.
# Any completed information will be lost. 

# To reset the stored answers uncomment and run these lines of code below. Re-comment the lines afterwards to avoid them running again. 
# utils.reset_checklist()
# checklist = utils.load_checklist()

# You can then re-run each section to reload it on the reset data. 

In [18]:

print("Dataset:", checklist["GeneralInformation"]["DatasetName"])
print("Dataset link:", checklist["GeneralInformation"]["DatasetLink"])
print("Assessor:", checklist["GeneralInformation"]["AssessorName"])
print("Assessor email:", checklist["GeneralInformation"]["AssessorEmailAddress"])

Dataset: 
Dataset link: 
Assessor: 
Assessor email: 


---

## **5. Data Preparation**

### Null values

In [19]:

dataset_null_values = widgets.Combobox(
            value=checklist['DataPreparation']['NullValuesFilled'],
            options=['Yes', 'No', 'N/A'],
            description='5.1 Have null values/gaps been filled?',
            placeholder=PLACEHOLDER,
            layout=widgets.Layout(width=WIDGET_WIDTH),
            style = DESCRIPTION_STYLE
            )

display(dataset_null_values)

Combobox(value='', description='5.1 Have null values/gaps been filled?', layout=Layout(width='900px'), options…

In [20]:
# This function will help you review if the dataset has missing or filled values. 
# However, it is still worth reading the dataset documentation as it should be explained in detail there. 

missing_value_stats = utils.find_missing_values(dataset)

print("MISSING / FILLED VALUE REPORT:")
for stats in missing_value_stats:
    for key, value in stats.items():
        print(f"    > {key}: {value}")
    print("-" * 50)

MISSING / FILLED VALUE REPORT:
    > variable_name: tas
    > missing_values_count: 1099749
    > percentage_missing: 20.223349839353176
    > has_fill_value: False
    > fill_value: None
    > filled_values_count: 0
    > percentage_filled: 0.0
--------------------------------------------------


### Outliers

In [21]:

dataset_outliers = widgets.Combobox(
            value=checklist['DataPreparation']['OutliersIdentified'],
            options=['Yes, tagged ', 'Yes, removed', 'No', 'N/A'],
            description='5.2 Have outliers been identified?',
            placeholder=PLACEHOLDER,
            layout=widgets.Layout(width=WIDGET_WIDTH),
            style = DESCRIPTION_STYLE
            )

display(dataset_outliers)


Combobox(value='', description='5.2 Have outliers been identified?', layout=Layout(width='900px'), options=('Y…

In [22]:
# This function could be used to review the z-score values over a set threshold. 
# This could be useful as an indication of outliers, however again, reading the dataset documentation should have information regarding outliers. 

outlier_stats = utils.count_z_score_outliers_for_dataset(dataset, threshold=3)

# Print the results for each variable
print("OUTLIER VALUE REPORT:")
for stats in outlier_stats:
    for key, value in stats.items():
        print(f"    > {key}: {value}")
    print("-" * 50)

OUTLIER VALUE REPORT:
    > variable_name: tas
    > total_values: 4338267
    > num_outliers: 84468
    > percentage_outliers: 1.9470447531237705
--------------------------------------------------


### Gridded data


In [23]:

dataset_gridded = widgets.Combobox(
            value=checklist['DataPreparation']['Gridded'],
            options=['Regularly gridded in space', 'Constant time-frequency', 'Regularly gridded in space and constant time-frequency', 'Not gridded', 'N/A'],
            description='5.3 Is the data gridded (regularly sampled in time and space)?',
            placeholder=PLACEHOLDER,
            layout=widgets.Layout(width=WIDGET_WIDTH),
            style = DESCRIPTION_STYLE
            )

dataset_gridded_transformed = widgets.Combobox(
            value=checklist['DataPreparation']['TransformedFromOriginal'],
            options=['Yes, from irregular sampling', 'Yes, from a different regular sampling', 'No, this is the original sampling', 'N/A'],
            description='If the data is gridded, was it transformed from a different original sampling?',            
            placeholder=PLACEHOLDER,
            layout=widgets.Layout(width=WIDGET_WIDTH),
            style = DESCRIPTION_STYLE
            )


dataset_gridded_original_sample = widgets.Combobox(
            value=checklist['DataPreparation']['OriginalSamplingAvailable'],
            options=['Yes', 'No', 'Only by request', 'N/A'],
            description = 'If the data is resampled from the original sampling, is the data also available at the original sampling?',
            placeholder=PLACEHOLDER,
            layout=widgets.Layout(width=WIDGET_WIDTH),
            style = DESCRIPTION_STYLE
            )

display(dataset_gridded, dataset_gridded_transformed, dataset_gridded_original_sample)

Combobox(value='', description='5.3 Is the data gridded (regularly sampled in time and space)?', layout=Layout…

Combobox(value='', description='If the data is gridded, was it transformed from a different original sampling?…

Combobox(value='', description='If the data is resampled from the original sampling, is the data also availabl…

In [28]:
# The functions and information found in notebook 1 (part 2) might prove useful to answer some of the questions above. 

resolution, coverage = gridded.get_spatial_resolution_and_coverage(dataset)
temporal_resolution, temporal_coverage = gridded.get_temporal_resolution_and_coverage(dataset)
spatial_consistency = gridded.check_spatial_consistency(dataset)
temporal_consistency = gridded.check_temporal_consistency(dataset)

print(f"Spatial resolution: Latitude {resolution[0]} deg, Longitude {resolution[1]} deg")
print(f"Spatial coverage: Latitude {coverage['latitude'][0]} to {coverage['latitude'][1]}, Longitude {coverage['longitude'][0]} to {coverage['longitude'][1]}")

print("Temporal resolution:", temporal_resolution)
print("Min time:", temporal_coverage['time'][0])
print("Max time:", temporal_coverage['time'][1])

print("Spatial resolution is consistent" if spatial_consistency == True 
      else "Spatial resolution not consistent" if spatial_consistency == False
      else "Unable to determin spatial resolution consistency")

print("Temporal resolution is consistent" if temporal_consistency == True 
      else "Temporal resolution not consistent" if temporal_consistency == False
      else "Unable to determin temporal resolution consistency")

Temporal resolution is not consistent.


Spatial resolution: Latitude 5.0 deg, Longitude 5.0 deg
Spatial coverage: Latitude -87.5 to 87.5, Longitude -177.5 to 177.5
Temporal resolution: 730
Min time: 1850-01-16T12:00:00.000000000
Max time: 2024-10-16T12:00:00.000000000
Spatial resolution is consistent
Temporal resolution not consistent


### Targets / labels for supervised learning

In [24]:

dataset_targets_or_labels = widgets.Combobox(
            value=checklist['DataPreparation']['SupervisedLearningLabels'],
            options=['Yes', 'No', 'N/A'],
            description = '5.4 Are there associated targets or labels for supervised learning techniques?',
            placeholder='Click to select option - (Can this be used as a training dataset)?',
            layout=widgets.Layout(width=WIDGET_WIDTH),
            style = DESCRIPTION_STYLE
            )

dataset_targets_or_labels_standards_label = widgets.Label(
    value = "If there are associated targets/labels, are community labeling standards implemented?"
)

dataset_targets_or_labels_standards = widgets.Text(
            value=checklist['DataPreparation']['SupervisedLearningLabelStandards'],
            placeholder = 'e.g., STAC label extension, ESA AIREO specification, etc.',
            layout = widgets.Layout(width=WIDGET_WIDTH)
)

display(dataset_targets_or_labels, dataset_targets_or_labels_standards_label, dataset_targets_or_labels_standards)

Combobox(value='', description='5.4 Are there associated targets or labels for supervised learning techniques?…

Label(value='If there are associated targets/labels, are community labeling standards implemented?')

Text(value='', layout=Layout(width='900px'), placeholder='e.g., STAC label extension, ESA AIREO specification,…

In [25]:

# Save button
save_button = widgets.Button(description="Save Data Access Answers to json file",  button_style="primary",  layout=widgets.Layout(flex='1 1 auto', width='auto'))

def generate_updates_preparation():

    updates = {
        "DataPreparation": {
            "NullValuesFilled": dataset_null_values.value,
            "OutliersIdentified": dataset_outliers.value,
            "Gridded": dataset_gridded.value,
            "TransformedFromOriginal": dataset_gridded_transformed.value,
            "OriginalSamplingAvailable": dataset_gridded_original_sample.value, 
            "SupervisedLearningLabels" : dataset_targets_or_labels.value,
            "SupervisedLearningLabelStandards" : dataset_targets_or_labels_standards.value,
          
        }
    }
    return updates

save_button.on_click(lambda b: utils.update_checklist(b, generate_updates_preparation()))

display(save_button)

Button(button_style='primary', description='Save Data Access Answers to json file', layout=Layout(flex='1 1 au…

## Finished

In [26]:

button_print_json = widgets.Button(description="Print json results",  button_style='info', layout=widgets.Layout(flex='1 1 auto', width='auto'))
output = widgets.Output()

display(button_print_json, output)

def print_json_info(b):
    """
    Loads a copy of the json file to checklist variable. 
    Then prints the json file contents to Jupyter notebook cell output.

    Arguments: b - represents the button calling the function. 
    """
    checklist = utils.load_checklist()
    with output:
        clear_output()
        for key, value in checklist.items():
            print(f"{key}:")
            if isinstance(value, dict):
                for sub_key, sub_value in value.items():
                    print(f"  {sub_key}: {sub_value}")
            else:
                print(f"  {value}")

button_print_json.on_click(print_json_info)


Button(button_style='info', description='Print json results', layout=Layout(flex='1 1 auto', width='auto'), st…

Output()

---

## **Appendix** - Definition of terms used in the checklist.